In [1]:
import torch
import functions
import modules

- Generates a training and a test set of 1,000 points sampled uniformly in [0, 1]2, each with a
label 0 if outside the disk of radius 1/√2π and 1 inside,
- two input units, two output units, three hidden layers of 25 units,

In [2]:
layer0 = modules.Linear(2, 25)
layer1 = modules.Linear(25, 25)
layer2 = modules.Linear(25, 25)
layer3 = modules.Linear(25, 25)
layer4 = modules.Linear(25, 2)

In [5]:
seq = modules.Sequential(layer0, layer1, layer2, layer3, layer4)

In [15]:
for name, m in seq.__dict__.items():
    print(name)
    print(m)

Linear0
Linear1
Linear2
Linear3
Linear4


In [18]:
import torch
import math

def generate_disc_set(nb):
    input = torch.Tensor(nb, 2).uniform_(0, 1)
    target = input.pow(2).sum(1).sub(1 / (2 * math.pi)).sign().add(1).div(2).long()
    return input, target

In [19]:
train_input, train_target = generate_disc_set(1000)